In [22]:

import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt

b_cancer = pd.read_csv("../Cases/Wisconsin/BreastCancer.csv", index_col=0)
lbl = LabelEncoder()
y = lbl.fit_transform( b_cancer['Class'] )
X = b_cancer.drop('Class', axis=1)

scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=24,stratify=y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)



print(X_torch.size())
print(y_torch.size())


# torch.manual_seed(24)
# model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=1))

torch.manual_seed(24)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=4),
                      nn.Tanh(),         
                      nn.Linear(in_features=4, out_features=1))

summary(model, (1,X_scl_trn.shape[1]))

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.4)
optimizer


for p in model.parameters():
    print(p)

y_pred = model(X_torch.float())
y_pred[:3]
    

y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)



for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())



torch.Size([489, 9])
torch.Size([489])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 4]              40
              Tanh-2                 [-1, 1, 4]               0
            Linear-3                 [-1, 1, 1]               5
Total params: 45
Trainable params: 45
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
Parameter containing:
tensor([[ 0.1763, -0.0833, -0.2833,  0.0205,  0.3107, -0.1487, -0.1085,  0.2607,
         -0.0464],
        [-0.1273, -0.0671,  0.0122, -0.2716, -0.0953,  0.3232, -0.0715,  0.3036,
         -0.1301],
        [-0.1341, -0.0994, -0.2980, -0.2008,  0.2015, -0.2501,  0.1139,  0.3136,
         -0.0424],
        [ 0

In [23]:
X_torch_test = torch.from_numpy(X_scl_tst)
### Inferencing on test set
lin_output = model(X_torch_test.float()) # Equivalent predict_proba / predict

np_out = lin_output.detach().numpy()
y_pred_prob = 1 / (1 + np.exp(-np_out))


y_pred_prob = y_pred_prob.reshape(y_test.shape[0],) 

y_pred = np.where(y_pred_prob >= 0.5,1,0)


print(accuracy_score(y_test,y_pred))

log_loss(y_test, y_pred_prob)

0.9714285714285714


0.08131366585486448